In [2]:
import sys,os
sys.path.insert(0, os.path.join(os.path.abspath(''), "YOLOX"))

import numpy as np
import cv2
import imutils
import time

from tracker import Tracker
from pose_detector import PoseDetector
from sequence_detector import SequenceDetector
from upscale import Upscaler
from visualizer import visualize

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
from pose_tracker import PoseTracker
tracker = Tracker('person')
pose_detector = PoseDetector(n_feature=8, n_output=3, checkpoint_path="pose_classification_simplified.ckpt", simple_mode=True, model_based=False)
sequence_detector = SequenceDetector([1,2], 10)
pose_tracker = PoseTracker(pose_detector, sequence_detector, tracker)
poi = None
upscaler = Upscaler("realesrgan_s", enhance_face=False)

In [7]:
state = 0
poi = None

while True:
    _, img = cap.read()
    if img is None:
        break

    ## Downsize from the Loomo robot
    im = imutils.resize(img, height=120)

    # Image preprocessing
    im = upscaler.enhance(im)
    im = imutils.resize(im, height=500, inter=cv2.INTER_BITS2)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    im = cv2.filter2D(im, -1, kernel)

    # FSM
    if state == 0:
        outputs, poi = pose_tracker.update(im)
        pose_in_progress = False
        for i,  p in pose_tracker.sequence_detector.people.items():
            if (p.i_seq > 0) and (i != poi):
                pose_in_progress = True
                break
        if (poi != None) and (not pose_in_progress):
            last_detected = time.time()
            state = 1
    if state == 1:
        check_poi = 0
        outputs = np.array(pose_tracker.tracker.update(im))
        for i in range(outputs.shape[0]) :
            if outputs[i,4] ==poi:
                check_poi = 1
        if (check_poi == 0) or (time.time()-last_detected >1):
            state = 0
    visualize(im, outputs, poi)


KeyboardInterrupt: 